Для начала приведем csv файл в корекнтный формат

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


input_file = "/content/drive/MyDrive/final_train.csv"
df = pd.read_csv(input_file)

In [ ]:
df.head()
len(df)

446559

In [ ]:
texts = df['text'].values
emojis_data = df['emoji'].factorize()
emojis = emojis_data[0]
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, emojis, test_size=0.1, random_state=42)

In [ ]:
emojis_data[0][:10]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 0])

In [ ]:
map = {}
for i in range(len(emojis_data[0])):
    em = emojis_data[0][i]
    if em not in map:
        map[em] = 1
    else:
        map[em] += 1

print(len(map))

62


In [ ]:
def emoji_to_num(emoji):
  for index, elem in enumerate(emojis_data[1]):
    if elem == emoji:
      return index

def num_to_emoji(num):
  return emojis_data[1][num]

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch


tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
tokenizer.truncation_side = 'left'
bert = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [ ]:
def tokenize_data(texts, tokenizer, max_len=24):
    return tokenizer(
        texts.tolist(),
        max_length=max_len,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )

In [ ]:
train_encodings = tokenize_data(train_texts, tokenizer)
val_encodings = tokenize_data(val_texts, tokenizer)

train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)


Создаем пайторч датасет

In [ ]:
import torch

class EmojiDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx].clone().detach()
        return item

train_dataset = EmojiDataset(train_encodings, train_labels)
val_dataset = EmojiDataset(val_encodings, val_labels)


In [ ]:
def tokenize_data2(texts, tokenizer, max_len=24):
    return tokenizer(
        texts,
        max_length=max_len,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
test = ["здарова sdgd sdgdfg",
        "привет"]
emb_test = tokenize_data2(test, tokenizer)
emb_test2 = bert(**emb_test)
emb_test2.last_hidden_state.shape

torch.Size([2, 13, 1024])

Создаем модель

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        probs = F.softmax(inputs, dim=-1)
        # Достаем нужные вероятности для классов, указанных в targets
        targets_one_hot = F.one_hot(targets, num_classes=inputs.shape[-1])
        probs = torch.sum(probs * targets_one_hot, dim=-1)

        # Вычисляем фокусирующий параметр
        focal_weight = (1 - probs) ** self.gamma

        # Стандартная кросс-энтропийная ошибка
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')

        loss = focal_weight * ce_loss

        if self.alpha is not None:
            alpha_weight = self.alpha.gather(0, targets)
            loss = alpha_weight * loss

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss


find alphas

In [ ]:

num_classes = 62
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

alpha = torch.zeros(num_classes).to(device)
for i in range(len(alpha)):
  alpha[i] = 8000 / map[i]


alpha2 = torch.ones(num_classes).to(device)

alpha3 = torch.sqrt(alpha)



In [ ]:
!pip install peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 8.4 MB/s eta 0:00:00


In [ ]:
import torch.nn as nn
from transformers import BertModel
from transformers import AutoModel
from peft import LoraConfig, get_peft_model
import torch.nn.functional as F

class EmojiClassifier2(nn.Module):
    def __init__(self, num_labels, alpha):
        super(EmojiClassifier2, self).__init__()
        self.bert = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

        lora_config = LoraConfig(
            r=16,
            lora_alpha=32,
            target_modules=["query", "value", "key"],
            bias="none"
        )

        self.bert = get_peft_model(self.bert, lora_config)
        self.alpha = alpha

        for param in self.bert.parameters():
            param.requires_grad = False
        for name, param in self.bert.named_parameters():
            if 'lora' in name:
                param.requires_grad = True


        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_labels)
        )

    def forward(self, input_ids, attention_mask, token_type_ids, labels=None): # тут посмотреть
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.pooler_output
        logits = self.classifier(cls_output)

        if labels is not None:
            loss_fct = FocalLoss(gamma=1.0, alpha=self.alpha)
            loss = loss_fct(logits, labels)
            return loss, logits
        return logits

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import LambdaLR


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = EmojiClassifier2(num_labels=len(df['emoji'].unique()), alpha=alpha2).to(device)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

optimizer = AdamW(model.parameters(), lr=2e-4)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def lr_lambda(current_step: int):
    warmup_steps = 1000  # Количество шагов для warmup
    if current_step < warmup_steps:
        return float(current_step) / float(max(1, warmup_steps))
    return 1.0

scheduler = LambdaLR(optimizer, lr_lambda=lr_lambda)


In [ ]:
from tqdm import tqdm
import torch
from tqdm import tqdm
import numpy as np

from torch.utils.tensorboard import SummaryWriter

def train_model(model, train_loader, val_loader, epochs=10):
    # Инициализация TensorBoard
    writer = SummaryWriter()

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        loop = tqdm(train_loader, leave=True)

        for batch in loop:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs[0]
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            loop.set_description(f'Epoch [{epoch+1}/{epochs}]')
            loop.set_postfix(loss=loss.item())

        # Логируем тренировочную потерю в TensorBoard
        avg_loss = total_loss / len(train_loader)
        writer.add_scalar('Loss/train', avg_loss, epoch)
        print(f'Epoch {epoch + 1}, Loss: {avg_loss}')

        model.eval()
        correct = 0
        total = 0
        correct_top10 = 0
        class_total = np.zeros(num_classes)
        class_correct = np.zeros(num_classes)
        class_predicted = np.zeros(num_classes) # num of elements that were predicted to this class


        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validating"):
                batch = {k: v.to(device) for k, v in batch.items()}
                logits = model(**batch)[1]
                predictions = torch.argmax(logits, dim=-1)
                correct += (predictions == batch['labels']).sum().item()
                total += len(batch['labels'])

                top10_preds = torch.topk(logits, k=10, dim=-1).indices
                correct_top10 += sum([batch['labels'][i] in top10_preds[i] for i in range(len(batch['labels']))])

                for i in range(len(batch['labels'])):
                    label = batch['labels'][i].item()
                    pred = predictions[i].item()
                    class_predicted[pred] += 1
                    class_total[label] += 1
                    if label == pred:
                        class_correct[label] += 1


        top1_acc = correct / total * 100
        top10_acc = correct_top10 / total * 100

        # Логируем в TensorBoard метрики валидации
        writer.add_scalar('Accuracy/Top-1', top1_acc, epoch)
        writer.add_scalar('Accuracy/Top-10', top10_acc, epoch)

        print(f'Validation Accuracy (Top-1): {top1_acc:.2f}%')
        print(f'Validation Accuracy (Top-10): {top10_acc:.2f}%')

        for i in range(num_classes):
            if class_total[i] > 0 and class_predicted[i] > 0:
                recall = 100 * class_correct[i] / class_total[i]
                prec = 100 * class_correct[i] / class_predicted[i]
                print(f'Class {i} Recall: {recall:.2f}% ({int(class_correct[i])}/{int(class_total[i])})      Precision: {prec:.2f}% ({int(class_correct[i])}/{int(class_predicted[i])})') # num of elements from this class that were predicted correctly
            else:
                print(f'Class {i} Accuracy: N/A (нет примеров для этого класса) class_predicted {class_predicted[i]}, class_total {class_total[i]}, class_correct {class_correct[i]}')

    # Закрываем TensorBoard
    writer.close()



train_model(model, train_loader, val_loader)

Epoch [1/10]: 100%|██████████| 1570/1570 [1:00:20<00:00,  2.31s/it, loss=2.88]


Epoch 1, Loss: 3.236215493177912


Validating: 100%|██████████| 698/698 [03:39<00:00,  3.18it/s]


Validation Accuracy (Top-1): 20.27%
Validation Accuracy (Top-10): 68.26%
Class 0 Recall: 47.14% (948/2011)      Precision: 22.66% (948/4183)
Class 1 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 389.0, class_correct 0.0
Class 2 Recall: 24.51% (274/1118)      Precision: 18.55% (274/1477)
Class 3 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 351.0, class_correct 0.0
Class 4 Recall: 5.36% (82/1531)      Precision: 14.49% (82/566)
Class 5 Recall: 0.30% (1/338)      Precision: 16.67% (1/6)
Class 6 Recall: 68.50% (3496/5104)      Precision: 22.10% (3496/15821)
Class 7 Recall: 22.84% (845/3699)      Precision: 13.85% (845/6099)
Class 8 Recall: 0.15% (1/678)      Precision: 100.00% (1/1)
Class 9 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 215.0, class_correct 0.0
Class 10 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 387.0, class_correct 0.0
Class 11 Recall: 4.5

Epoch [2/10]: 100%|██████████| 1570/1570 [59:42<00:00,  2.28s/it, loss=2.95]


...

Epoch 3, Loss: 2.8879368548180646


Validating: 100%|██████████| 698/698 [03:39<00:00,  3.18it/s]


Validation Accuracy (Top-1): 21.10%
Validation Accuracy (Top-10): 70.49%
Class 0 Recall: 46.84% (942/2011)      Precision: 23.56% (942/3998)
Class 1 Recall: 0.00% (0/389)      Precision: 0.00% (0/5)
Class 2 Recall: 25.94% (290/1118)      Precision: 21.15% (290/1371)
Class 3 Recall: 0.85% (3/351)      Precision: 17.65% (3/17)
Class 4 Recall: 9.01% (138/1531)      Precision: 14.51% (138/951)
Class 5 Recall: 2.66% (9/338)      Precision: 12.68% (9/71)
Class 6 Recall: 72.69% (3710/5104)      Precision: 22.09% (3710/16792)
Class 7 Recall: 13.65% (505/3699)      Precision: 14.80% (505/3413)
Class 8 Recall: 10.03% (68/678)      Precision: 18.23% (68/373)
Class 9 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 215.0, class_correct 0.0
Class 10 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 387.0, class_correct 0.0
Class 11 Recall: 4.56% (85/1863)      Precision: 15.98% (85/532)
Class 12 Recall: 0.21% (2/936)      Precision: 20.00% 

Epoch [4/10]:  60%|██████    | 949/1570 [36:37<23:57,  2.32s/it, loss=2.93]


KeyboardInterrupt: 

In [ ]:
from tqdm import tqdm
import torch
from tqdm import tqdm
import numpy as np

from torch.utils.tensorboard import SummaryWriter

def val_model(model, train_loader, val_loader, epochs=10):

    model.eval()
    correct = 0
    total = 0
    correct_top10 = 0
    class_total = np.zeros(num_classes)
    class_correct = np.zeros(num_classes)
    class_predicted = np.zeros(num_classes) # num of elements that were predicted to this class


    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            batch = {k: v.to(device) for k, v in batch.items()}
            logits = model(**batch)[1]
            predictions = torch.argmax(logits, dim=-1)
            correct += (predictions == batch['labels']).sum().item()
            total += len(batch['labels'])

            top10_preds = torch.topk(logits, k=10, dim=-1).indices
            correct_top10 += sum([batch['labels'][i] in top10_preds[i] for i in range(len(batch['labels']))])

            for i in range(len(batch['labels'])):
                label = batch['labels'][i].item()
                pred = predictions[i].item()
                class_predicted[pred] += 1
                class_total[label] += 1
                if label == pred:
                    class_correct[label] += 1


    top1_acc = correct / total * 100
    top10_acc = correct_top10 / total * 100


    print(f'Validation Accuracy (Top-1): {top1_acc:.2f}%')
    print(f'Validation Accuracy (Top-10): {top10_acc:.2f}%')

    for i in range(num_classes):
        if class_total[i] > 0 and class_predicted[i] > 0:
            recall = 100 * class_correct[i] / class_total[i]
            prec = 100 * class_correct[i] / class_predicted[i]
            print(f'Class {i} Recall: {recall:.2f}% ({int(class_correct[i])}/{int(class_total[i])})      Precision: {prec:.2f}% ({int(class_correct[i])}/{int(class_predicted[i])})') # num of elements from this class that were predicted correctly
        else:
            print(f'Class {i} Accuracy: N/A (нет примеров для этого класса) class_predicted {class_predicted[i]}, class_total {class_total[i]}, class_correct {class_correct[i]}')



val_model(model, train_loader, val_loader)

Validating: 100%|██████████| 698/698 [03:40<00:00,  3.16it/s]

Validation Accuracy (Top-1): 21.30%
Validation Accuracy (Top-10): 70.79%
Class 0 Recall: 50.77% (1021/2011)      Precision: 23.15% (1021/4410)
Class 1 Recall: 0.77% (3/389)      Precision: 15.79% (3/19)
Class 2 Recall: 23.79% (266/1118)      Precision: 21.43% (266/1241)
Class 3 Recall: 1.42% (5/351)      Precision: 16.13% (5/31)
Class 4 Recall: 9.41% (144/1531)      Precision: 16.40% (144/878)
Class 5 Recall: 4.14% (14/338)      Precision: 12.61% (14/111)
Class 6 Recall: 75.43% (3850/5104)      Precision: 21.79% (3850/17670)
Class 7 Recall: 14.92% (552/3699)      Precision: 14.07% (552/3923)
Class 8 Recall: 9.00% (61/678)      Precision: 24.02% (61/254)
Class 9 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 215.0, class_correct 0.0
Class 10 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 387.0, class_correct 0.0
Class 11 Recall: 9.50% (177/1863)      Precision: 14.39% (177/1230)
Class 12 Recall: 0.00% (0/936)      Precision

In [ ]:
bert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

total_params = sum(p.numel() for p in bert.parameters())
trainable_params = sum(p.numel() for p in bert.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")





In [ ]:

for param in bert.parameters():
    param.requires_grad = False

total_params = sum(p.numel() for p in bert.parameters())
trainable_params = sum(p.numel() for p in bert.parameters() if p.requires_grad)


print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

In [ ]:
bert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "value"],
    bias="none"
)

bert = get_peft_model(bert, lora_config)

for param in bert.parameters():
    param.requires_grad = False


for name, param in bert.named_parameters():
            if 'lora' in name:
                param.requires_grad = True

total_params = sum(p.numel() for p in bert.parameters())
trainable_params = sum(p.numel() for p in bert.parameters() if p.requires_grad)


print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")


Total parameters: 178148352
Trainable parameters: 294912


In [ ]:
bert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "value", "key"],
    bias="none"
)

bert = get_peft_model(bert, lora_config)

for param in bert.parameters():
    param.requires_grad = False


for name, param in bert.named_parameters():
            if 'lora' in name:
                param.requires_grad = True

total_params = sum(p.numel() for p in bert.parameters())
trainable_params = sum(p.numel() for p in bert.parameters() if p.requires_grad)


print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")


Total parameters: 178295808
Trainable parameters: 442368


In [ ]:
bert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["query", "value", "key"],
    bias="none"
)

bert = get_peft_model(bert, lora_config)

for param in bert.parameters():
    param.requires_grad = False


for name, param in bert.named_parameters():
            if 'lora' in name:
                param.requires_grad = True

total_params = sum(p.numel() for p in bert.parameters())
trainable_params = sum(p.numel() for p in bert.parameters() if p.requires_grad)


print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")


Total parameters: 179622912
Trainable parameters: 1769472
